In [1]:
import logging
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from lmflux import (
    openai_agent, tool,
    Session, Agent, Message,
    Templates
)
from lmflux.graphs import MeshGraph

# Creating a new tool
mock_employee = {
    "123456789": {"name": "Caio", "status": "ACTIVE", "department": "Engineering"},
    "987654321": {"name": "Frederico", "status": "INACTIVE", "department": "Sales"},
    "111111111": {"name": "Joao", "status": "ACTIVE", "department": "Accounting"},
}
@tool
def get_information(employee_id: str) -> int:
    """Gets any necessary data from HR database"""
    data = mock_employee.get(employee_id, {"error": "Employee with this ID is not found!"})
    return data

# Create an agent
peter = openai_agent(
    "peter", 
    "openai.gpt_oss_20b", tools=[],
    system_prompt="You are Peter, an AI agent you directly comunicate with other agents to get the information you need."
)
bob = openai_agent(
    "bob", 
    "openai.gpt_oss_20b", 
    tools=[get_information], 
    system_prompt="You are Bob, an AI agent, you have access to employee records and helps other agents."
)

In [3]:
G = MeshGraph()
G.connect_agents(peter, bob, "You can ask bob things about employees")
G.show_mermaid()

```mermaid
graph TB
f6f1665b-fadd-43c9-9126-1f455d0698be(peter)
c2b53696-a562-4548-ac42-bd2856f95c6d(bob)
f6f1665b-fadd-43c9-9126-1f455d0698be --Can call--> c2b53696-a562-4548-ac42-bd2856f95c6d
```

In [5]:
message = G.query_agent(peter, "What is the status of employee id 123456789?", show_progress=True)

**Human**:

What is the status of employee id 123456789?

**Agent Calls**

- peter (5 messages)
	- bob (5 messages)


In [7]:
G.set_mermaid_render()
G.show_result()

```mermaid
flowchart TB
    %% Global wrapper for all agents
    subgraph ALL_AGENTS["All Agents"]
        direction TB
        subgraph subgraph_peter["peter"]
            msg0["system"]
            msg1["user"]
            msg0 --> msg1
            msg2["assistant"]
            msg1 --> msg2
            msg3["tool"]
            msg2 --> msg3
            msg4["assistant"]
            msg3 --> msg4
            c879952a-47a1-4c23-a96e-97773979a1b2__out(("talk to bob"))
            msg2 --> c879952a-47a1-4c23-a96e-97773979a1b2__out
        end
        subgraph subgraph_bob["bob"]
            msg5["system"]
            msg6["user"]
            msg5 --> msg6
            msg7["assistant"]
            msg6 --> msg7
            msg8["tool"]
            msg7 --> msg8
            msg9["assistant"]
            msg8 --> msg9
            c879952a-47a1-4c23-a96e-97773979a1b2__in(("talk to peter"))
            c879952a-47a1-4c23-a96e-97773979a1b2__in --> msg6
            msg9 --> c879952a-47a1-4c23-a96e-97773979a1b2__in
        end
    end
    %% Global wrapper for humans
    subgraph subgraph_human["Human"]
        hmsg10["User Query"]
        hmsg11["User Response"]
    end
    class c879952a-47a1-4c23-a96e-97773979a1b2__in interaction
    class c879952a-47a1-4c23-a96e-97773979a1b2__out interaction
c879952a-47a1-4c23-a96e-97773979a1b2__out <-.-> c879952a-47a1-4c23-a96e-97773979a1b2__in
        hmsg10 -.-> msg1
        msg4 -.-> hmsg11
```